In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import time
# from google.colab import drive
import os
import gc
# def mount():
#   drive.mount('/content/drive')
#   os.chdir("/content/drive/My Drive/SLAss2")
# mount()

In [3]:
# train = pd.read_csv("dtrain123.dat", delimiter = "  ", header = None)
test = pd.read_csv("data/dtest123.dat", delimiter = "  ", header = None, engine = "python")
train = pd.read_csv("data/zipcombo.dat", delimiter = " ", header = None, engine = "python")
train.shape
minitrain = pd.read_csv("data/dtrain123.dat", delimiter = "  ", header = None, engine = "python")
train = np.array(train)
xtrain = train[:,1:257]
ytrain = train[:,0]
one = np.array(minitrain)
xone = one[:, 1:257]
yone = one[:,0]
test = np.array(test)
xtest = test[:, 1:]
ytest = test[:,0]
def split_data(fullK, jtrain, jtest, y):
    Ktrain = fullK[jtrain][:,jtrain].copy()
    Ktest =  fullK[jtrain][:,jtest].copy()
    ytrain = y[jtrain]
    ytest = y[jtest]
    return Ktrain, ytrain, Ktest, ytest


In [1]:
import numpy as np
import time
import pandas as pd
from sklearn.metrics.pairwise import polynomial_kernel as polykernel
test = pd.read_csv("data/dtest123.dat", delimiter = "  ", header = None, engine = "python")
train = pd.read_csv("data/zipcombo.dat", delimiter = " ", header = None, engine = "python")
train.shape
minitrain = pd.read_csv("data/dtrain123.dat", delimiter = "  ", header = None, engine = "python")
train = np.array(train)
xtrain = train[:,1:257]
ytrain = train[:,0]
one = np.array(minitrain)
xone = one[:, 1:257]
yone = one[:,0]
test = np.array(test)
xtest = test[:, 1:]
ytest = test[:,0]
class kernel_perceptron:
    def __init__(self, N):
        self.N = N
        self.alpha = np.zeros((self.N, 1))
        self.accs = []
        self.w = np.zeros((N, 1))
    def train(self, K, y, d = 2, epochs = 1, alt = True):
        if len(self.alpha) != len(K):
            self.N = len(K)
            self.alpha = np.zeros((len(K), 1))
        for epoch in range(epochs):
            js = np.arange(self.N)
            np.random.shuffle(js)
            for i in js:
                yhat = np.sign(np.sign((K[:,i].dot(self.alpha))))
                if yhat != y[i]:
                    self.alpha[i] += y[i]

    def predict(self, Ktest):
        return Ktest.T.dot(self.alpha).flatten()

class onevsall:
    def __init__(self, xtrain, ytrain, kernel = "poly", kargs = None):
        self.N = len(xtrain)
        self.models = []
        if kernel == "poly":
        	self.K = polykernel(xtrain, degree = 2)
        self.y = ytrain
    
    def train(self):
        self.vals = range(int(max(self.y)+1))
        if len(self.models) == 0:
            self.ytemps = []
            for val in self.vals:
                ytemp = self.y.copy()
                ytemp[ytemp != val] = -1
                ytemp[ytemp == val] = 1
                self.add_model(ytemp)
                self.ytemps.append(ytemp.copy())
        for val in self.vals:
            self.models[val].train(self.K, self.ytemps[val], epochs = 1)

    def add_model(self, ytrain):
        model = kernel_perceptron(self.N)
        self.models.append(model)
    
    def predict(self, k, full = False, test = True):
        preds = np.zeros((k.shape[1], len(self.models)))
        yhat = np.zeros(k.shape[1])
        for i, model in enumerate(self.models):
            preds[:,i] = model.predict(k)
        if full == True:
            return preds
        return preds.argmax(axis = 1)
yone[yone==3] = 0
model = onevsall(xone, yone)
for i in range(10):
	model.train()
print(model.predict(xone), yone)
print(np.mean(model.predict(xone).flatten() == yone.flatten()))

[1 1 1 0 0 2 2 1 1 0 0 0 0 1 1 1 1 2 1 2 2 2 2 1 1 0 0 0 0 0 1 1 1 1 2 2 2
 2 2 1 1 0 0 0 0 1 1 1 1 1 2 2 2 2 2 1 1 2 0 0 1 0 0 1 1 1 2 2 2 0 0 1 1 0
 0 0 1 0 0 1 1 1 2 2 2 0 0 1 1 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 2 2 2 2 2 1 1 0 0 0 0 0 1 1 2 2 2 2
 2 2 2 2 1 2 0 0 0 0 0 1 2 2 2 2 2 2 2 2 1 2 2 0 0 0 0 0 2 2 2 2 2 2 2 1 1
 2 2 0 0 0 0 0 2 2 2 2 2 2 2 1 1 1 0 0 0 0 0 0 1 2 2 2 2 2 1 1 1 0 0 0 0 2
 2 0 1 2 2 2 2 2 0 1 0 0 0 0 0 2 2 1 1 1 2 2 2 2 0 0 0 0 0 0 2 2 1 1] [0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 2. 1. 0. 0. 2. 2. 2. 2. 2. 1. 2. 0.
 0. 2. 2. 2. 1. 1. 0. 0. 1. 2. 2. 2. 1. 1. 2. 1. 1. 0. 0. 1. 2. 0. 0. 2.
 2. 1. 2. 1. 2. 2. 1. 2. 2. 1. 0. 2. 0. 2. 2. 2. 1. 2. 2. 2. 2. 2. 1. 2.
 2. 2. 1. 1. 1. 2. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 2. 1.
 2. 1. 2. 2. 1. 0. 2. 0. 2. 0. 2. 1. 2. 0. 1. 1. 0. 2. 1. 2. 1. 1. 2. 2.
 1. 1. 2. 2. 2. 2. 1. 0. 1. 2. 1. 2. 0. 1. 2. 2. 1. 1. 2. 2. 1. 1. 0. 1.
 2. 2. 1. 1. 2. 2. 2. 2. 1. 2. 1. 1. 1. 0.

<ipython-input-1-633a3b900226>:77: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  print(np.mean(model.predict(xone).flatten() == yone.flatten()))


In [8]:
polykernel(xone).shape

(329, 329)

In [5]:
model.predict(xone).shape

(256,)

In [4]:
yone.shape

(329,)

In [8]:
class kernel_perceptron:
    def __init__(self, N):
        self.N = N
        self.alpha = np.zeros((self.N, 1))
        self.accs = []
        self.w = np.zeros((N, 1))
    def train(self, K, y, d = 2, epochs = 1, alt = True):
        if len(self.alpha) != len(K):
            self.N = len(K)
            self.alpha = np.zeros((len(K), 1))
        for epoch in range(epochs):
            mistakes = 0
            yhats = np.zeros(self.N)
            js = np.arange(self.N)
            np.random.shuffle(js)
            for i in js:
                yhat = np.sign(np.sign((K[:,i].dot(self.alpha))))
                if yhat != y[i]:
                    self.alpha[i] += y[i]
                    mistakes += 1
                # else:
                #     self.alpha[js[i]] = 0
                yhats[i] = yhat
            self.accs.append(1 - mistakes/self.N)
        return yhats

    
    def predict(self, Ktest):
        return Ktest.T.dot(self.alpha).flatten()
                
def K(p, q, d = 2):
    return (p.T.dot(q))**d

def GaussK(p, q, c):
    print(np.sum((p-q)**2))
    return np.exp(-c * np.sum((p-q)**2))

class onevsall:
    def __init__(self, xtrain, ytrain, xtest = None, ytest = None, kernel = K, d = 2, ks = None):
        self.N = len(xtrain)

        self.models = []
        self.accs = []
        self.testaccs = []
        self.d = d
        self.x, self.xtest = xtrain, xtest
        self.y, self.ytest = ytrain, ytest
        if ks == None:
            self.K = np.zeros((self.N, self.N))
            for i in range(self.N):
                for j in range(i, self.N):
                    # print(self.x[i], self.x[j], d)
                    self.K[i,j] = K(self.x[i], self.x[j], d)
            self.K = self.K + self.K.T - np.diag(self.K.diagonal())
            self.Ktest = np.zeros((self.N, len(self.xtest)))
            for i in range(self.N):
                for j in range(len(self.xtest)):
                    self.Ktest[i,j] = K(self.x[i], self.xtest[j], d)
        else:
            self.N = len(ks[0])
            self.K, self.Ktest = ks
    
    def train(self, print_accs = True, alt = False):
        self.vals = range(int(max(self.y)+1)) # final yval doesn't need model
        if len(self.models) == 0:
            self.ytemps = []
            for val in self.vals:
                ytemp = self.y.copy()
                ytemp[ytemp != val] = -1
                ytemp[ytemp == val] = 1
                self.add_model(ytemp)
                self.ytemps.append(ytemp.copy())
        for val in self.vals:
            self.models[val].train(self.K, self.ytemps[val], epochs = 1, alt = alt)

        if print_accs:
            yhat = self.predict(self.K, self.y)
            acc = np.mean(yhat.reshape(len(yhat), 1) == self.y.reshape(len(yhat), 1))
            self.accs.append(acc)            
            yhattest = self.predict(self.Ktest, self.ytest)
            testacc = np.mean(yhattest.reshape(len(yhattest), 1) == self.ytest.reshape(len(yhattest), 1))
            self.testaccs.append(testacc)
            print(f"Training Accuracy: {acc}. Testing Accuracy: {testacc}")

    def add_model(self, ytrain):
        model = kernel_perceptron(self.N)
        # model.train(self.K, ytrain, epochs = epochs, d = self.d)
        self.models.append(model)
    
    def predict(self, k, full = False):
        preds = np.zeros((k.shape[1], len(self.models)))
        yhat = np.zeros(k.shape[1])
        for i, model in enumerate(self.models):
            preds[:,i] = model.predict(k)
        if full == True:
            return preds
        return preds.argmax(axis = 1)




In [14]:
class onevsone:
    
    def __init__(self, xtrain, ytrain, xtest = None, ytest = None, kernel = K, d = 2, ks = None):
        self.N = len(xtrain)

        self.models = []
        self.accs = []
        self.testaccs = []
        self.d = d
        self.x, self.xtest = xtrain, xtest
        self.y, self.ytest = ytrain, ytest
        if ks == None:
            self.K = np.zeros((self.N, self.N))
            for i in range(self.N):
                for j in range(i, self.N):
                    # print(self.x[i], self.x[j], d)
                    self.K[i,j] = K(self.x[i], self.x[j], d)
            self.K = self.K + self.K.T - np.diag(self.K.diagonal())
            self.Ktest = np.zeros((self.N, len(self.xtest)))
            for i in range(self.N):
                for j in range(len(self.xtest)):
                    self.Ktest[i,j] = K(self.x[i], self.xtest[j], d)
        else:
            self.N = len(ks[0])
            self.K, self.Ktest = ks
            
    
    def train(self, print_accs = True, alt = False):
        self.vals = range(int(max(self.y)+1))
        self.m = int(max(self.y)+1)
        m = self.m
        if len(self.models) == 0:
            self.Ks = [int(max(self.y+1))*[None] for i in range(int(max(self.y+1)))]
            self.ijs = [int(max(self.y+1))*[None] for i in range(int(max(self.y+1)))]
            self.ys = [int(max(self.y+1))*[None] for i in range(int(max(self.y+1)))]
            self.models = [int(max(self.y+1))*[None] for i in range(int(max(self.y+1)))]
            for i in range(0, m):
                for j in range(1+i, m):
                    ij = np.where((self.y==i)|(self.y==j))[0]
                    self.models[i][j] = kernel_perceptron(len(ij))
                    ij = np.where((self.y==i)|(self.y==j))[0]
                    ktemp = self.K[ij, :]
                    ktemp = ktemp[:,ij]
                    ktemp = ktemp.squeeze()
                    ytemp = self.y[ij]
                    ytemp[ytemp == i] = 1
                    ytemp[ytemp == j] = -1
                    self.Ks[i][j] = ktemp.copy()
                    self.ys[i][j] = ytemp.copy()
                    self.ijs[i][j] = ij
        for i in range(0, m):
            for j in range(1+i, m):
                self.models[i][j].train(self.Ks[i][j], self.ys[i][j])

    
    def predict(self, K):
        n = K.shape[1]
        preds = np.zeros((n, self.m, self.m)) 
        for i in range(0, self.m):
            for j in range(i+1, self.m):
                preds[:,i,j] = model.models[i][j].predict(K[self.ijs[i][j],:])
        scores = np.zeros((n, 10))
        for n, pred in enumerate(preds):
            for i in range(10):
                scores[n,i] += -np.sum(pred[:i, i]) + np.sum(pred[i,i:])
        return np.argmax(scores, axis = 1)

TypeError: object of type 'NoneType' has no len()

## 1.1
Average of 20 runs w/ 80/20 split for train and test error

In [17]:
ntrain = int(0.1*len(xtrain))
ntrain = 7439
metrics = np.zeros((7, 20, 2))
start = time.time()
for d in range(4, 0, -1):
    print(f"DDDDDD: {d}")
    fullKd = np.load(f"Gram Matrices/K{d}")

    for i in range(1):
        js = np.arange(len(xtrain))
        np.random.shuffle(js)
        jtrain = js[:ntrain]
        jtest = js[ntrain:ntrain+1000]
        # jtrain = np.arange(0, 5000)
        # rng = np.random.default_rng()
        Ktrain, ytraini, Ktest, ytesti = split_data(fullKd, jtrain, jtest, ytrain)
        model = onevsone(xtrain, ytraini, ks = (Ktrain, Ktest))
        for epoch in range(10):
            model.train()
        yhat = model.predict(Ktest)
        metrics[d-1,i] = [np.mean(ytraini.reshape(len(ytraini), 1) == model.predict(Ktrain).reshape(len(ytraini), 1)),
                          np.mean(ytesti.reshape(len(ytesti), 1) == yhat.reshape(len(ytesti), 1))]
        print(d, i)
        print(metrics[d-1, i])
        np.save(open("1.6metrics1.1", "wb"), metrics)
        if i==0:
            print(f"Time Taken: {time.time() - start}")

DDDDDD: 4
4 0
[0.99099341 0.957     ]
Time Taken: 18.94263005256653
DDDDDD: 3
3 0
[0.98158355 0.945     ]
Time Taken: 38.19206953048706
DDDDDD: 2
2 0
[0.96558677 0.956     ]
Time Taken: 56.86452293395996
DDDDDD: 1
1 0
[0.87081597 0.838     ]
Time Taken: 75.4625985622406


# 1.2

In [ ]:
ntrain = int(0.8*len(xtrain))
metrics = np.zeros((7, 20, 2))
nfold = int(0.8 * ntrain)
nfoldtest = int(ntrain - nfold)
dstars = np.zeros(20)
testerr = np.zeros(20)
confusion_matrix = np.zeros((10, 10))
print("Start")
start = time.time()
for i in range(20):
    np.random.seed(i+10)
    js = np.arange(len(xtrain))
    np.random.shuffle(js)
    jtrain = js[:ntrain]
    jtest = js[ntrain:]
    # jtrain = np.arange(0, 5000)
    # rng = np.random.default_rng()
    bestacc = 0
    dstar = 7
    for d in range(1, 8):

        # split into 80/20
        fullKd = np.load(f"K{d}")
        Ktrain, ytraini, Ktest, ytesti = split2(fullKd, jtrain, jtest, ytrain, ntrain)
        # five-fold cross validation
        testacc = 0
        for k in range(5):
            nsplit = int(nfold * k/5)
            ktrain1 = np.arange(0, nsplit+1)
            ktest = np.arange(nsplit+1, nsplit + nfoldtest-1)
            ktrain2 = np.arange(nsplit + nfoldtest-1, nfold)
            ktrain = np.hstack([ktrain1, ktrain2]).flatten()
            Kfoldtrain, yfoldtraini, Kfoldtest, yfoldtesti = split2(Ktrain, ktrain, ktest, ytraini, nfold)
            model = onevsall(yfoldtraini, yfoldtraini, None, yfoldtesti, d = d, ks = (Kfoldtrain, Kfoldtest))
            for epoch in range(10):
                model.train(False, False)    
            yhat = model.predict(Kfoldtest, yfoldtesti)
            testacc += 0.2 * np.mean(yhat.flatten() == yfoldtesti.flatten())
        print(f"{i} run : {d}: {testacc}, {bestacc}, {dstar}")
        if testacc > bestacc:
            bestacc = testacc
            dstar = d
#             yhat = model.predict(Ktest)
#             testerr[i] = 1 - np.mean(yhat.flatten() == ytesti.flatten())
    #train final model for run
    fullKd = np.load(f"K{dstar}")
    Ktrainfinal, ytraini, Ktestfinal, ytesti = split2(fullKd, jtrain, jtest, ytrain, ntrain)
    finalmodel = onevsall(ytrain, ytraini, None, ytesti, ks = (Ktrainfinal, Ktestfinal))
    for epoch in range(10):
        finalmodel.train(False, False)
    finalyhat = finalmodel.predict(Ktestfinal, ytesti)
    dstars[i] = dstar
    testerr[i] = 1 - (finalmodel.predict(finalmodel.Ktest, finalmodel.ytest).flatten() == finalmodel.ytest.flatten()).mean()
    print(f"{i} runs finished. Dstar: {dstar}, Testerr: {testerr[i]}, Time: {time.time() -start}")
    
    # confusion matrix
    confi = np.zeros((10, 10))
    for m in range(len(yhat)):
        confi[int(ytesti[m]), int(finalyhat[m])] += 1
    confusion_matrix += confi
#         metrics[d-1,i] = [np.mean(ytraini.reshape(len(ytraini), 1) == model.predict(Ktrain).reshape(len(ytraini), 1))
#                           , np.mean(ytesti.reshape(len(ytesti), 1) == yhat.reshape(len(ytesti), 1))]
#         print(d, i)
#         print(metrics[d-1, i])
#     np.save(open("dstars", "wb"), dstars)
#     np.save(open("12testerr", "wb"), testerr)
#     np.save(open("conf", "wb"), confusion_matrix)

Start
0 run : 1: 0.9153432032301482, 0, 7
0 run : 2: 0.96164199192463, 0.9153432032301482, 1
0 run : 3: 0.9652759084791387, 0.96164199192463, 2
0 run : 4: 0.9699865410497982, 0.9652759084791387, 3
0 run : 5: 0.9694481830417228, 0.9699865410497982, 4
0 run : 6: 0.9694481830417228, 0.9699865410497982, 4
0 run : 7: 0.9679676985195156, 0.9699865410497982, 4
0 runs finished. Dstar: 4, Testerr: 0.01935483870967747, Time: 808.0366711616516
1 run : 1: 0.9193808882907133, 0, 7
1 run : 2: 0.9612382234185732, 0.9193808882907133, 1
1 run : 3: 0.9660834454912517, 0.9612382234185732, 2
1 run : 4: 0.9682368775235533, 0.9660834454912517, 3
1 run : 5: 0.9690444145356663, 0.9682368775235533, 4
1 run : 6: 0.9683714670255721, 0.9690444145356663, 5
1 run : 7: 0.9672947510094214, 0.9690444145356663, 5
1 runs finished. Dstar: 5, Testerr: 0.027956989247311825, Time: 1610.322770357132
2 run : 1: 0.9145356662180351, 0, 7
2 run : 2: 0.9602960969044415, 0.9145356662180351, 1
2 run : 3: 0.964064602960969, 0.960296